In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors
import skimage.color as color
from ipywidgets import VBox, HBox, FloatLogSlider
from scipy.stats import entropy

# For importing from alternative directory sources
import sys  
sys.path.insert(0, '../dip_utils')


from huff_utils import (build_huff_tree,
                        build_huff_encoder,
                        build_huff_pair,
                        load_huffable_image,
                        test_tree_making)
from matrix_utils import (arr_info,
                          make_linmap)
from vis_utils import (vis_rgb_cube,
                       vis_hsv_cube,
                       vis_hists,
                       vis_pair,
                       lab_uniform)

# Test Encoding/Decoding

In [2]:
I = load_huffable_image('../dip_pics/happy128.png')
encoder, decoder = build_huff_pair(I)
enI = ''.join(encoder[pix] for pix in I.ravel())

loadHuffableImage: input is multi-channel, using grayscale.
loadHuffableImage: Setting range to [0, 255]


In [3]:
I.ravel()

array([255, 255, 255, ..., 255, 255, 255], dtype=uint8)

In [4]:
def decode_image (huffStr,decoder):
    dec = []
    while huffStr:
        for k in decoder:
            if huffStr.startswith(k):
                dec.append(decoder[k])
                huffStr = huffStr[len(k):]
    res = np.asarray(dec, dtype=np.uint8)
    return res

In [5]:
deI= decode_image(enI,decoder)

In [6]:
deI

array([255, 255, 255, ..., 255, 255, 255], dtype=uint8)

In [7]:
np.all(deI == I.ravel())

True

In [8]:
arr_info(I)

((128, 128), dtype('uint8'), 0, 255)

In [9]:
reconstruct_I = deI.reshape(128,128)

In [10]:
vis_pair(I,reconstruct_I)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
I2 = plt.imread('../dip_pics/coral.jpg')

In [12]:
R,G,B = np.split(I2, indices_or_sections=3,axis=-1)

In [13]:
print("R: ", arr_info(R))
print("G: ", arr_info(G))
print("B: ", arr_info(B))

R:  ((297, 448, 1), dtype('uint8'), 0, 255)
G:  ((297, 448, 1), dtype('uint8'), 0, 255)
B:  ((297, 448, 1), dtype('uint8'), 0, 255)


In [14]:
encR, decR = build_huff_pair(R)
encG, decG = build_huff_pair(G)
encB, decB = build_huff_pair(B)
enR = ''.join(encR[pix] for pix in R.ravel())
enG = ''.join(encG[pix] for pix in G.ravel())
enB = ''.join(encB[pix] for pix in B.ravel())

In [15]:
reconstructed_I2 = np.dstack((decode_image(enR,decR).reshape(297,448),decode_image(enG,decG).reshape(297,448),decode_image(enB,decB).reshape(297,448)))

In [16]:
print(arr_info(I2))
print(arr_info(reconstructed_I2))

((297, 448, 3), dtype('uint8'), 0, 255)
((297, 448, 3), dtype('uint8'), 0, 255)


In [17]:
np.all(I2.ravel() == reconstructed_I2.ravel())

True

In [18]:
vis_pair(I2,reconstructed_I2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
def reset(R,G,B):
    Rset = 0
    Gset = 0
    Bset = 0
    
    if R.ravel().max() <= 1:
        Rset = 1
        R1 = np.round(256*R).astype('uint8')
    else:
        R1 = R
    
    if G.ravel().max() <= 1:
        Gset = 1
        G1 = np.round(256*G).astype('uint8')
    else:
        G1 = G

    if B.ravel().max() <= 1:
        Bset = 1
        B1 = np.round(256*B).astype('uint8')
    else:
        B1 = B        
    
    return R1,G1,B1,Rset,Gset,Bset

In [20]:
def getEntropy(R,G,B):
    binsR = np.arange(257)
    freqR, binsR = np.histogram(R.ravel(), bins=binsR)
    entR = entropy(freqR, base=2)
    
    binsG = np.arange(257)
    freqG, binsG = np.histogram(G.ravel(), bins=binsG)
    entG = entropy(freqG, base=2)
    
    binsB = np.arange(257)
    freqB, binsB = np.histogram(B.ravel(), bins=binsB)
    entB = entropy(freqB, base=2)
    
    return entR,entG,entB

In [21]:
def getsize8bit(R,G,B):
    sizeR = 8*np.prod(R.shape)
    sizeG = 8*np.prod(G.shape)
    sizeB = 8*np.prod(B.shape)
    return sizeR,sizeG,sizeB

In [22]:
def printStatSingleChannel(channel,reset,en,ent,size,bpp):
    print('Channel' + str(channel) + 'statistics:')
    if reset == 1:
        print('LoadHuffableImage: Setting range to [0, 255]')
    print(str(channel) + 'channel entropy is ' + str(ent) + 'bits.')
    print('Size at 8-bit encoding:     ' + "{:.2f}".format(size/8000) + 'KB.')
    print('Size with huff encoding:     ' + "{:.2f}".format(len(en)/8000) + "KB, or " + "{:.2f}".format(bpp) + " per pixel.")
    print("---------------------------------------------------------")

In [23]:
def getCompressionStats(imageFileName):
    I = plt.imread(imageFileName)
    R,G,B = np.split(I,indices_or_sections=3,axis=-1)
    
    R2,G2,B2,Rset,Gset,Bset = reset(R,G,B)
        
    #encoding and decoding dictionaries
    encR, decR = build_huff_pair(R2)
    encG, decG = build_huff_pair(G2)
    encB, decB = build_huff_pair(B2)
    
    # Size with huff encoding: len(enR/G/B)
    enR = ''.join(encR[pix] for pix in R2.ravel())
    enG = ''.join(encG[pix] for pix in G2.ravel())
    enB = ''.join(encB[pix] for pix in B2.ravel())
    
    # Entropy value
    entR,entG,entB = getEntropy(R2,G2,B2)
    
    # Original Size
    sizeR,sizeG,sizeB = getsize8bit(R2,G2,B2)
    
    # Bits per pixel actual use
    bppR = len(enR)/np.prod(R2.shape)
    bppG = len(enG)/np.prod(G2.shape)
    bppB = len(enB)/np.prod(B2.shape)
    
    # Print results
    printStatSingleChannel('Red',Rset,enR,entR,sizeR,bppR)
    printStatSingleChannel('Green',Gset,enG,entG,sizeG,bppG)
    printStatSingleChannel('Blue',Bset,enB,entB,sizeB,bppB)

In [24]:
getCompressionStats('../dip_pics/happy128.png')

ChannelRedstatistics:
LoadHuffableImage: Setting range to [0, 255]
Redchannel entropy is 3.4173796912298164bits.
Size at 8-bit encoding:     16.38KB.
Size with huff encoding:     7.12KB, or 3.48 per pixel.
---------------------------------------------------------
ChannelGreenstatistics:
LoadHuffableImage: Setting range to [0, 255]
Greenchannel entropy is 3.4058077464419907bits.
Size at 8-bit encoding:     16.38KB.
Size with huff encoding:     7.09KB, or 3.46 per pixel.
---------------------------------------------------------
ChannelBluestatistics:
LoadHuffableImage: Setting range to [0, 255]
Bluechannel entropy is 3.308168417838215bits.
Size at 8-bit encoding:     16.38KB.
Size with huff encoding:     6.89KB, or 3.36 per pixel.
---------------------------------------------------------


In [25]:
test = plt.imread('../dip_pics/3D_shape.jpg')
plt.figure()
plt.imshow(test)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
getCompressionStats('../dip_pics/3D_shape.jpg')

ChannelRedstatistics:
Redchannel entropy is 6.01930101544606bits.
Size at 8-bit encoding:     9794.57KB.
Size with huff encoding:     7400.44KB, or 6.04 per pixel.
---------------------------------------------------------
ChannelGreenstatistics:
Greenchannel entropy is 5.821433684037192bits.
Size at 8-bit encoding:     9794.57KB.
Size with huff encoding:     7167.77KB, or 5.85 per pixel.
---------------------------------------------------------
ChannelBluestatistics:
Bluechannel entropy is 4.467539866672991bits.
Size at 8-bit encoding:     9794.57KB.
Size with huff encoding:     5508.34KB, or 4.50 per pixel.
---------------------------------------------------------
